In [2]:
import pandas as pd
import numpy as np

In [3]:
business=pd.read_csv("./arizona_business_details.csv")

In [4]:
business.shape

(11072, 16)

In [3]:
ids=business['business_id']

In [5]:
ids.head()

0    8-NRKkPY1UiFXW20WXKiXg
1    _c3ixq9jYKxhLUB0czi0ug
2    3ByGQOVgds2YEu6kzl-XEQ
3    CeuTRtwsq6w5rztGOyNMPg
4    AcGRSWCpb7YB95MTsHlGEw
Name: business_id, dtype: object

In [ ]:
#reviews=pd.read_csv("yelp_dataset/yelp_academic_dataset_review.json")
with open('yelp_dataset/yelp_academic_dataset_review.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 
df=pd.DataFrame(data)

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import os 
files=os.listdir("yelp_dataset/reviews/")

In [9]:
reviews_df=None

In [10]:
reviews_df=pd.DataFrame()
for file in files:
    print(file)
    df=pd.read_json("yelp_dataset/reviews/"+file,lines=True)
    merged=pd.merge(business,df,on='business_id')
    reviews_df=reviews_df.append(merged, ignore_index=True)

xax
xce
xba
xcg
xac
xbg
xas
xbl
xbc
xby
xaz
xbt
xao
xan
xal
xam
xbh
xbf
xbw
xbz
xbo
xcd
xae
xad
xbp
xaf
xbu
xbm
xah
xau
xcb
xbs
xbk
xaq
xay
xbe
xak
xbq
xav
xbr
xar
xaa
xap
xai
xbb
xat
xbd
xab
xaw
xcf
xbn
xbx
xca
xch
xag
xaj
xbi
xbv
xbj
xcc


In [12]:
reviews_backup=reviews_df.copy()

In [6]:
reviews_backup.to_csv("arizona_reviews.csv")

NameError: name 'reviews_backup' is not defined

In [7]:
reviews_backup=pd.read_csv("arizona_reviews.csv")

In [9]:
reviews_df=reviews_backup[['business_id','review_id','text','stars_y']]

In [122]:
reviews_df.to_csv('review_arizon.csv')

In [10]:
reviews_df.columns

Index(['business_id', 'review_id', 'text', 'stars_y'], dtype='object')

In [12]:
np.mean(reviews_df.groupby('business_id').sum())

stars_y    355.227511
dtype: float64

## Topic modling over reviews for a restaurant

In [3]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
from gensim.summarization import summarize
from gensim.summarization import keywords
import pandas as pd
import numpy as np

In [4]:
reviews_df=pd.read_csv('review_arizon.csv')

Few of the below methods have been taken from the API's [tutoria](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/). 

In [6]:
def clean(doc):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [7]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

def getReviewForTopic(ldamodel,corpus,texts):
    df_topic_sents_keywords=format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=texts)
    sent_topics_sorteddf_mallet = pd.DataFrame()
    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                                 grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                                axis=0)
    # Reset Index    
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
    # Format
    sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
    return sent_topics_sorteddf_mallet

In [23]:
def getTopics(restuarant_reviews,n):
    reviews=[rev for rev in restuarant_reviews['text']]
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    reviews_clean = [clean(review).split() for review in reviews]        
    dictionary = corpora.Dictionary(reviews_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    review_term_matrix = [dictionary.doc2bow(review) for review in reviews_clean]
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(review_term_matrix, num_topics=n, id2word = dictionary, passes=50)
    topics=[]
    if ldamodel.num_topics-1<n:
        n=ldamodel.num_topics-1
    ldamodel
    for i in range(0, n):
        topics.append(ldamodel.print_topic(i))
    #also return summary of relevant reviews for each topic 
    topic_reviews=getReviewForTopic(ldamodel=ldamodel,corpus=review_term_matrix,texts=reviews)
    topic_reviews_text=list(topic_reviews['Text'])
    review_summary=[]
    for rev in topic_reviews_text:
        review_summary.append(summarize(rev, word_count = 20))
        #review_summary.append(rev)
    #coherence_model_lda = CoherenceModel(model=ldamodel, texts=topic_reviews_text, dictionary=dictionary, coherence='c_v')
    #coherence_lda = coherence_model_lda.get_coherence()
    #print('\nCoherence Score: ', coherence_lda)
    return topics,review_summary
    
def getRestuarantTopics(business_id,n=5):
    reviews_df=pd.read_csv('review_arizon.csv')
    positive_reviews=reviews_df[reviews_df.business_id==business_id][reviews_df.stars_y>=3]
    negative_reviews=reviews_df[reviews_df.business_id==business_id][ reviews_df.stars_y<3]
    positive_topics,pos_reviews=getTopics(positive_reviews,n)
    negative_topics,neg_reviews=getTopics(negative_reviews,n)
    return positive_topics,pos_reviews,negative_topics,neg_reviews

In [10]:
reviews=[]
for _,rev in reviews_df[reviews_df.business_id=='_c3ixq9jYKxhLUB0czi0ug'].iterrows():
    review=rev['text']
    reviews.append(review)


In [55]:
df=pd.read_csv("phoenix_business_ws_rw_ffall_merged2.csv")
df.tail(13)

,zipcode,business_id,Mexican,American (Traditional),Pizza,American (New),Burgers,Italian,Chinese,Salad,...,median_income,median_age,under_18,above_18,walkscore,ffall,stars,review_count,ffall_category,CuisineCombined
2812,85014,9yESUcIVarhYEabIMj844Q,0,0,0,1,0,0,0,0,...,43682,36.2,5299,19202,60.0,1651,4.0,557,5,4
2813,85014,tIOjJWfu4Dqz-FIzXHRvCg,0,0,0,0,0,0,0,1,...,43682,36.2,5299,19202,60.0,162,4.5,105,2,8
2814,85029,niqIaJy9UXu-DS7BIUvUoA,0,0,1,0,0,0,0,0,...,42224,33.2,11210,31624,41.0,232,3.5,84,2,3
2815,85014,rMtphJbzFYA3eLx-AT62vA,0,1,0,0,0,0,0,0,...,43682,36.2,5299,19202,60.0,556,3.0,100,4,2
2816,85040,_7hhpjQHxjPi0gNzzMmO6g,0,0,0,0,0,0,1,0,...,32133,27.5,10096,19186,44.0,4,2.0,7,0,7
2817,85013,I5CKfLZ-hxriv5IzHLvNGQ,0,0,0,1,0,0,0,0,...,50182,38.9,3718,15358,62.0,4,4.5,3,0,28
2818,85021,yjUgQXvycpnH9-lfBA2sgA,1,0,0,0,0,0,0,0,...,37595,33.1,9712,27481,51.0,54,3.0,16,1,19
2819,85013,JQYkVnUzS5vgFxZgCuW7ag,0,0,0,0,0,0,0,0,...,50182,38.9,3718,15358,62.0,1207,4.5,535,5,19
2820,85014,5l8VDB6hXXm-uJP2_4SRMw,0,0,0,1,0,0,0,1,...,43682,36.2,5299,19202,60.0,73,3.0,28,1,12
2821,85042,jIDnLUpdKQCYtvT7wT7mJg,0,1,0,0,1,0,0,0,...,46676,30.8,12063,28422,35.0,76,3.5,24,1,7


In [56]:
count=1
word='burger'
for _,rev in reviews_df[reviews_df.business_id=='9yESUcIVarhYEabIMj844Q'].iterrows():
    review=rev['text']
    if word in review:
        #print(summarize(rev, word_count = 20))
        print(review,rev['review_id'])    

I would definitely recommend. The restaurant has not atmosphere and is cool and trendy. I had the burger with havarti cheese and bacon - Delish! The burger was moist and juicy and the bread it came on was so fresh and tasty. I also had the grilled asparagus, but did not really like it served cold, so next time may ask that they heat it up. All in all, it is a must try. fxU4mWGl2-FLUhd5W4ncUg
Great service from all and a really fun atmosphere. Honestly we came to play ping pong and enjoy a few drinks but decided to order dinner. I sampled the shaved prime rib dip sandwich that a friend ordered, it was very nice with perfectly rosey prime rib. I loved my pub burger and fries. Fries were hot, crispy and flavorful and my burger was one of the best I've had. Next time we go to the valley we know where we'll eat. -s0jRzP3HGflL8ckXQYUEA
I'm not planning to come back, but wouldn't say no if someone else wanted to go.

I've visited twice now with mixed experiences. I thought the food was as exp

In [57]:
from gensim.models import CoherenceModel

pos_topics,pos_reviews,neg_topics,neg_reviews=getRestuarantTopics('9yESUcIVarhYEabIMj844Q',4)

/home/kireeti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/kireeti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Printing extracted topics

In [58]:
pos_topics

['0.013*"place" + 0.011*"food" + 0.009*"great" + 0.007*"good" + 0.007*"drink" + 0.007*"like" + 0.006*"service" + 0.006*"pretzel" + 0.006*"bar" + 0.006*"back"',
 '0.016*"good" + 0.012*"place" + 0.011*"food" + 0.010*"great" + 0.010*"really" + 0.009*"time" + 0.008*"pretzel" + 0.006*"game" + 0.006*"restaurant" + 0.006*"drink"',
 '0.017*"great" + 0.013*"food" + 0.012*"place" + 0.008*"friend" + 0.007*"atmosphere" + 0.007*"time" + 0.007*"drink" + 0.007*"good" + 0.006*"service" + 0.006*"one"']

In [59]:
pos_reviews

['SERVICE - 3.5 stars...pretty good given the number of people flooding into the place',
 'The crusty bread was good but I really enjoyed dipping it in my main dish.',
 'I was not overwhelmed with this dish, and would have preferred a healthier serving a pears and more sauce to add additional sweetness.',
 'I had only 1 cocktail - the Maria Pepino, which is basically a bloody mary with tequila instead of vodka, and garnished with a pickled pepper.']

In [60]:
neg_topics

['0.017*"food" + 0.011*"drink" + 0.010*"service" + 0.010*"time" + 0.009*"back" + 0.008*"place" + 0.007*"minute" + 0.007*"get" + 0.006*"hour" + 0.006*"u"',
 '0.012*"food" + 0.010*"service" + 0.008*"like" + 0.007*"good" + 0.007*"server" + 0.005*"chicken" + 0.005*"great" + 0.005*"cheese" + 0.005*"place" + 0.005*"first"',
 '0.011*"restaurant" + 0.009*"u" + 0.009*"food" + 0.007*"hour" + 0.006*"people" + 0.006*"frustrating" + 0.006*"want" + 0.005*"party" + 0.005*"it" + 0.005*"table"']

In [61]:
neg_reviews

["I didn't exactly expect a ten minute turn around on our food; but really, almost an hour to deliver these dishes is seriously extreme.",
 'After waiting 40 minutes and the people who sat after us got their apps we decided to ask.',
 'Try to focus on the neighborhood locals; they are the ones who will keep you healthy.',
 "We had to wait awhile for the new steak and the waiter guy never brought me my next cocktail so I sat there with a dry throat picking at the boyfriend's salad."]